In [1]:
import time

from sklearn.tree import DecisionTreeRegressor

from keras.models import Sequential
from keras.layers import Dense

from sklearn.preprocessing import  MinMaxScaler
from sklearn import preprocessing

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import numpy as np
import csv
import os

/home/shashwati/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
sc= MinMaxScaler()

In [3]:
# function returns the data in the right format
def get_data():
    # load training dataset
    dataset = np.genfromtxt("YearPredictionMSD.csv", dtype='float', delimiter=",")
    
    train_X = dataset[0:463715,1:91]
    train_Y = dataset[0:463715,0]

    test_X = dataset[463715:,1:91]
    test_Y = dataset[463715:,0]

    return train_X, train_Y, test_X, test_Y

In [4]:
# builds the decision tree of depth 13
def regression_tree(train, label):
    dt = DecisionTreeRegressor(max_depth=13, min_samples_leaf=5000)
    dt.fit(train, label)
    return dt

In [5]:
# builds the neural network for a given class
def neural_network(class_data):
    num_train = []
    num_label = []
    for x in class_data:
        num_train.append(x[0])
        num_label.append(x[1])
    
    num_train = np.array(num_train)
    num_label = np.array(num_label)
    
    
    # Scale the features so they have 0 mean
    num_train = preprocessing.scale(num_train)
    
    num_label = num_label.reshape(-1,1)
    num_label = sc.fit_transform(num_label)
    
    model = Sequential()
    model.add(Dense(90, input_dim=90, kernel_initializer='normal', activation='relu'))
    model.add(Dense(90, kernel_initializer='normal', activation='relu'))
    model.add(Dense(90, kernel_initializer='normal', activation='relu'))
    model.add(Dense(90, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','accuracy'])
    model.fit(num_train, num_label, epochs=3, batch_size=32)
    
    return model 

In [6]:
# builds the neural shrub
def neural_shrubs(tree, train, label):
    train = np.array(train)
    label = np.array(label)
  
    # leave_id: index of the leaf that cantains the instance
    leave_id = tree.apply(train) 

    classes = dict()

    for x in range(len(train)):
        leaf = leave_id[x]
        
        # Gets the class for each leaf
        #.value: contains value of all the tree nodes
        #.value[leaf]: returns the value of the leaf
        #idx = tree.tree_.value[leaf][0][0]
        
        # insert the instance into the class
        if leaf in classes.keys():
            classes[leaf].append([train[x], label[x]])
        else:
            classes[leaf] = [[train[x], label[x]]]
    
    # stores the neural network for each class
    nn_models = dict()
    
    #stores the max time taken to build a neural network
    max_time = 0;

    for key in classes.keys():
        
        start = time.time()
        model = neural_network(classes[key])
        end = time.time()
        
        time_taken = end - start
        if max_time < time_taken:
            max_time = time_taken
        
        nn_models[key] = model

    # returns a neural network for each class and the max 
    # time taken to build the neural network
    return nn_models, max_time

In [7]:
# The algorithm to build the decision tree
train, train_label, test, test_label = get_data()

dt_start = time.time()
tree = regression_tree(train, train_label)
dt_end = time.time()
print("Decision tree made in: ", dt_end-dt_start)

Decision tree made in:  32.71749544143677


In [14]:
# Neural shrub
shrubs, max_time = neural_shrubs(tree, train, train_label)

Epoch 1/3
5023/5023 [==============================] - 5s 912us/step - loss: 0.0942 - mean_absolute_error: 0.2069 - acc: 0.0332
Epoch 2/3
5023/5023 [==============================] - 1s 250us/step - loss: 0.0174 - mean_absolute_error: 0.0972 - acc: 0.0380
Epoch 3/3
5023/5023 [==============================] - 1s 254us/step - loss: 0.0122 - mean_absolute_error: 0.0778 - acc: 0.0380
Epoch 1/3
9213/9213 [==============================] - 6s 598us/step - loss: 0.0612 - mean_absolute_error: 0.1730 - acc: 0.0230
Epoch 2/3
9213/9213 [==============================] - 2s 236us/step - loss: 0.0198 - mean_absolute_error: 0.1022 - acc: 0.0242
Epoch 3/3
9213/9213 [==============================] - 2s 243us/step - loss: 0.0175 - mean_absolute_error: 0.0941 - acc: 0.0242
Epoch 1/3
6667/6667 [==============================] - 5s 725us/step - loss: 0.0650 - mean_absolute_error: 0.1791 - acc: 0.0124
Epoch 2/3
6667/6667 [==============================] - 2s 236us/step - loss: 0.0188 - mean_absolute_erro

7138/7138 [==============================] - 2s 278us/step - loss: 0.0173 - mean_absolute_error: 0.0975 - acc: 0.0154
Epoch 3/3
7138/7138 [==============================] - 2s 261us/step - loss: 0.0144 - mean_absolute_error: 0.0877 - acc: 0.0154
Epoch 1/3
8951/8951 [==============================] - 6s 638us/step - loss: 0.0596 - mean_absolute_error: 0.1587 - acc: 0.0315
Epoch 2/3
8951/8951 [==============================] - 2s 264us/step - loss: 0.0094 - mean_absolute_error: 0.0698 - acc: 0.0336
Epoch 3/3
8951/8951 [==============================] - 2s 260us/step - loss: 0.0070 - mean_absolute_error: 0.0582 - acc: 0.0336
Epoch 1/3
6685/6685 [==============================] - 5s 769us/step - loss: 0.0710 - mean_absolute_error: 0.1893 - acc: 0.0129
Epoch 2/3
6685/6685 [==============================] - 2s 267us/step - loss: 0.0247 - mean_absolute_error: 0.1203 - acc: 0.0138
Epoch 3/3
6685/6685 [==============================] - 2s 271us/step - loss: 0.0219 - mean_absolute_error: 0.1127 

8932/8932 [==============================] - 3s 302us/step - loss: 0.0219 - mean_absolute_error: 0.1148 - acc: 0.0081
Epoch 1/3
5022/5022 [==============================] - 6s 1ms/step - loss: 0.1030 - mean_absolute_error: 0.2421 - acc: 0.0167
Epoch 2/3
5022/5022 [==============================] - 1s 299us/step - loss: 0.0418 - mean_absolute_error: 0.1613 - acc: 0.0199
Epoch 3/3
5022/5022 [==============================] - 2s 345us/step - loss: 0.0372 - mean_absolute_error: 0.1501 - acc: 0.0201
Epoch 1/3
5434/5434 [==============================] - 6s 1ms/step - loss: 0.0789 - mean_absolute_error: 0.1946 - acc: 0.0208
Epoch 2/3
5434/5434 [==============================] - 2s 288us/step - loss: 0.0202 - mean_absolute_error: 0.1048 - acc: 0.0232
Epoch 3/3
5434/5434 [==============================] - 2s 292us/step - loss: 0.0157 - mean_absolute_error: 0.0892 - acc: 0.0232
Epoch 1/3
7998/7998 [==============================] - 7s 854us/step - loss: 0.0656 - mean_absolute_error: 0.1832 - ac

6954/6954 [==============================] - 7s 1ms/step - loss: 0.0678 - mean_absolute_error: 0.1762 - acc: 0.0141
Epoch 2/3
6954/6954 [==============================] - 2s 334us/step - loss: 0.0164 - mean_absolute_error: 0.0929 - acc: 0.0155
Epoch 3/3
6954/6954 [==============================] - 3s 362us/step - loss: 0.0134 - mean_absolute_error: 0.0814 - acc: 0.0155
Epoch 1/3
7894/7894 [==============================] - 7s 941us/step - loss: 0.0658 - mean_absolute_error: 0.1671 - acc: 0.0329
Epoch 2/3
7894/7894 [==============================] - 3s 336us/step - loss: 0.0104 - mean_absolute_error: 0.0737 - acc: 0.0357
Epoch 3/3
7894/7894 [==============================] - 3s 326us/step - loss: 0.0082 - mean_absolute_error: 0.0635 - acc: 0.0357
Epoch 1/3
7006/7006 [==============================] - 7s 1ms/step - loss: 0.0784 - mean_absolute_error: 0.1895 - acc: 0.0253
Epoch 2/3
7006/7006 [==============================] - 2s 336us/step - loss: 0.0157 - mean_absolute_error: 0.0891 - ac

In [15]:
# training time
total_time = dt_end - dt_start + max_time
print("Training time: ", round(total_time, 5))

Training time:  47.95906


In [16]:
# predicts using the neural shrub
def neural_shrub_predict(tree, nn_model, test):
    test = np.asarray(test)
    
    pred = []
    
    # getting the predicted value
    tree_pred = tree.apply(test)
    
    test = preprocessing.scale(test)
    for i in range(len(test)):
        
        # gets the neural network for the tree predicted val
        x = tree_pred[i]
        nn_model_class = nn_model[x]
        
        # preprocessing
        l = np.asarray(test[i])
        l = l.reshape(-1, 90)
        
        # gets the val using the nn associated with
        # the current predicted value
        ans = nn_model_class.predict(l)
        pred.append(ans)
    
    return pred

In [17]:
# predicting for test data
# results scaled between 0 - 1

pred = neural_shrub_predict(tree, shrubs, test)

In [18]:
pred = np.asarray(pred)
pred = pred.ravel()
pred = pred.reshape(-1,1)
pred = sc.inverse_transform(pred)
    
print(mean_absolute_error(test_label, pred), mean_squared_error(test_label, pred))

6.7961220344692705 99.47642204472457


In [19]:
print(test_label[0:10])
print(pred[0:10])

[2007. 2003. 2005. 2003. 2005. 2007. 2003. 2003. 2003. 2005.]
[[1996.5908]
 [1999.8171]
 [2002.4985]
 [2005.3513]
 [2005.3563]
 [1999.4575]
 [1998.3212]
 [1994.4529]
 [1998.7758]
 [1999.3042]]
